In [1]:
import utils
import Simple_CNN
import Hierarchical

import torch
from torchvision import transforms
from torch.utils.data import DataLoader, random_split


import matplotlib.pyplot as plt

import numpy as np
from importlib import reload

import yaml

In [2]:
with open("hierarchy.yaml", "r") as f:
  net_params = yaml.load(f, Loader=yaml.FullLoader)

In [3]:
net_organization = net_params["net_organization"]

net_organization

{'phylum': {'class_Arthropoda': ['family_Arachnida', 'family_Insecta'],
  'class_Chordata': ['family_Amphibia',
   'family_Aves',
   'family_Mammalia',
   'family_Reptilia']}}

In [4]:
node_params = net_params["node_params"]
node_params

{'phylum': {'epochs': 5,
  'lr': 0.001,
  'order': [],
  'log_file': 'test_train.log'},
 'class_Arthropoda': {'epochs': 5,
  'lr': 0.001,
  'order': ['Arthropoda'],
  'log_file': 'test_train.log'},
 'family_Arachnida': {'epochs': 5,
  'lr': 0.001,
  'order': ['Arthropoda', 'Arachnida'],
  'log_file': 'test_train.log'},
 'family_Insecta': {'epochs': 5,
  'lr': 0.001,
  'order': ['Arthropoda', 'Insecta'],
  'log_file': 'test_train.log'},
 'class_Chordata': {'epochs': 5,
  'lr': 0.001,
  'order': ['Chordata'],
  'log_file': 'test_train.log'},
 'family_Actinopterygii': {'epochs': 5,
  'lr': 0.001,
  'order': ['Chordata', 'Actinopterygii'],
  'log_file': 'test_train.log'},
 'family_Amphibia': {'epochs': 5,
  'lr': 0.001,
  'order': ['Chordata', 'Amphibia'],
  'log_file': 'test_train.log'},
 'family_Aves': {'epochs': 5,
  'lr': 0.001,
  'order': ['Chordata', 'Aves'],
  'log_file': 'test_train.log'},
 'family_Mammalia': {'epochs': 5,
  'lr': 0.001,
  'order': ['Chordata', 'Mammalia'],
  'log_

In [5]:
net_architecture = net_params["net_architecture"]

net_architecture

{'phylum': {'input_shape': [3, 1024, 256],
  'conv_channels': [8, 16, 32, 16, 8, 4],
  'conv_kernel_size': [2, 2, 2, 2, 2, 2],
  'pooling_size': [[2, 2], [2, 2], [2, 2], [2, 2], [2, 2], [2, 2]],
  'conv_dropout': [0.0, 0.0, 0.2, 0.2, 0.0, 0.0],
  'linear_features': [64, 2],
  'linear_dropout': [0.4],
  'last_layer_activation': 'Sigmoid()'},
 'class_Arthropoda': {'input_shape': [3, 1024, 256],
  'conv_channels': [8, 16, 32, 16, 8, 4],
  'conv_kernel_size': [2, 2, 2, 2, 2, 2],
  'pooling_size': [[2, 2], [2, 2], [2, 2], [2, 2], [2, 2], [2, 2]],
  'conv_dropout': [0.0, 0.0, 0.2, 0.2, 0.0, 0.0],
  'linear_features': [64, 2],
  'linear_dropout': [0.4],
  'last_layer_activation': 'Sigmoid()'},
 'family_Arachnida': {'input_shape': [3, 1024, 256],
  'conv_channels': [8, 16, 32, 16, 8, 4],
  'conv_kernel_size': [2, 2, 2, 2, 2, 2],
  'pooling_size': [[2, 2], [2, 2], [2, 2], [2, 2], [2, 2], [2, 2]],
  'conv_dropout': [0.0, 0.0, 0.2, 0.2, 0.0, 0.0],
  'linear_features': [64, 1],
  'linear_dropout':

In [6]:
# Parameters
random_seed = 42

batch_size = 8
lr = 0.001
epochs = 100

net_name = "SimpleCNN2out_Drop_2"

In [7]:
reload(utils)

transform_img = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5),(0.5, 0.5, 0.5))
    ]
)

transform_labels = transforms.Compose(
    [transforms.ToTensor()]
)

dataset = utils.HierarchicalDataset("../datasets/smallDatasetBroad.csv",
                                    "../spectrograms/small_dataset_broad",
                                    "gbifID",
                                    ["phylum", "class", "family", "genus"],
                                    transform=transform_img)
dataset.csv

,gbifID,species,genus,family,class,phylum,file_name
0,1230246828,Pycnonotus nigricans,Pycnonotus,Pycnonotidae,Aves,Chordata,Chordata/Aves/Pycnonotidae/Pycnonotus/Pycnonot...
1,779853992,Emberiza hortulana,Emberiza,Emberizidae,Aves,Chordata,Chordata/Aves/Emberizidae/Emberiza/Emberiza_ho...
2,779849670,Cervus elaphus,Cervus,Cervidae,Mammalia,Chordata,Chordata/Mammalia/Cervidae/Cervus/Cervus_elaph...
3,1269848168,Numenius phaeopus,Numenius,Scolopacidae,Aves,Chordata,Chordata/Aves/Scolopacidae/Numenius/Numenius_p...
4,779860246,Neofelis nebulosa,Neofelis,Felidae,Mammalia,Chordata,Chordata/Mammalia/Felidae/Neofelis/Neofelis_ne...
...,...,...,...,...,...,...,...
1995,779852839,Dryobates minor,Dryobates,Picidae,Aves,Chordata,Chordata/Aves/Picidae/Dryobates/Dryobates_mino...
1996,991883728,Coccothraustes coccothraustes,Coccothraustes,Fringillidae,Aves,Chordata,Chordata/Aves/Fringillidae/Coccothraustes/Cocc...
1997,779865264,Rallus aquaticus,Rallus,Rallidae,Aves,Chordata,Chordata/Aves/Rallidae/Rallus/Rallus_aquaticus...
1998,1229953095,Colinus virginianus,Colinus,Odontophoridae,Aves,Chordata,Chordata/Aves/Odontophoridae/Colinus/Colinus_v...


In [8]:
dataset.set_order(["Arthropoda", "Arachnida"])
dataset[10]

{'image': tensor([[[-0.4667, -0.4667, -0.4667,  ..., -0.4667, -0.4667, -0.4667],
          [-0.4667, -0.4667, -0.4667,  ..., -0.4667, -0.4667, -0.4667],
          [-0.4667, -0.4667, -0.4667,  ..., -0.4667, -0.4667, -0.4667],
          ...,
          [-0.5608, -0.4275, -0.4431,  ..., -0.4431, -0.3725, -0.6314],
          [-0.5373, -0.4980, -0.5216,  ..., -0.5216, -0.4588, -0.5608],
          [-0.4588, -0.4980, -0.5216,  ..., -0.5216, -0.4431, -0.3882]],
 
         [[-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          ...,
          [-0.8980, -0.5686, -0.2078,  ..., -0.1137, -0.2863, -0.6706],
          [-1.0000, -0.8745, -0.7333,  ..., -0.7255, -0.8039, -0.9765],
          [-0.9922, -1.0000, -0.9765,  ..., -0.9922, -1.0000, -1.0000]],
 
         [[-0.3490, -0.3490, -0.3490,  ..., -0.3490, -0.3490, -0.3490],
          [-0.3490,

In [9]:
len(dataset)

2000

In [10]:
testing = DataLoader(dataset, batch_size=batch_size)

In [11]:
len(testing.dataset)

2000

In [51]:
out = next(iter(testing))
x, label = out["image"], out["label"]

x.shape, label.shape

(torch.Size([8, 3, 256, 1024]), torch.Size([8, 1]))

In [52]:
label

tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]])

In [56]:
torch.cuda.empty_cache()
reload(Hierarchical)
H = Hierarchical.HierarchicalClassification(net_architecture, node_params, net_organization, dataset)

#H.fit()

CUDA available: converting network to CUDA
CUDA available: converting network to CUDA
CUDA available: converting network to CUDA
CUDA available: converting network to CUDA
CUDA available: converting network to CUDA
CUDA available: converting network to CUDA
CUDA available: converting network to CUDA
CUDA available: converting network to CUDA
CUDA available: converting network to CUDA
Label: Musophagidae with probability 18.373%
Label: Acrididae with probability 14.776%
Label: Lycosidae with probability 12.496%
Label: Acrididae with probability 14.776%
Label: Lycosidae with probability 12.496%
Label: Acrididae with probability 14.776%
Label: Acrididae with probability 14.776%
Label: Acrididae with probability 14.776%


In [15]:
dataset.childs_of

[{'Chordata': 'Aves'},
 {'Chordata': 'Mammalia'},
 {'Arthropoda': 'Insecta'},
 {'Chordata': 'Amphibia'},
 {'Arthropoda': 'Arachnida'},
 {'Chordata': 'Reptilia'},
 {'Aves': 'Pycnonotidae'},
 {'Aves': 'Emberizidae'},
 {'Mammalia': 'Cervidae'},
 {'Aves': 'Scolopacidae'},
 {'Mammalia': 'Felidae'},
 {'Aves': 'Turdidae'},
 {'Aves': 'Corvidae'},
 {'Aves': 'Phylloscopidae'},
 {'Mammalia': 'Hominidae'},
 {'Aves': 'Spheniscidae'},
 {'Aves': 'Anatidae'},
 {'Aves': 'Acrocephalidae'},
 {'Aves': 'Picidae'},
 {'Aves': 'Certhiidae'},
 {'Aves': 'Strigidae'},
 {'Aves': 'Sylviidae'},
 {'Aves': 'Sturnidae'},
 {'Aves': 'Locustellidae'},
 {'Aves': 'Oriolidae'},
 {'Aves': 'Tytonidae'},
 {'Aves': 'Musophagidae'},
 {'Aves': 'Motacillidae'},
 {'Aves': 'Cisticolidae'},
 {'Mammalia': 'Tapiridae'},
 {'Aves': 'Ptilonorhynchidae'},
 {'Aves': 'Alaudidae'},
 {'Aves': 'Fringillidae'},
 {'Mammalia': 'Bovidae'},
 {'Aves': 'Rallidae'},
 {'Aves': 'Muscicapidae'},
 {'Insecta': 'Acrididae'},
 {'Aves': 'Bucerotidae'},
 {'Aves

In [ ]:
a = np.amax(test["name"], 1)
b = np.amax(test["name"], 1) + 1

In [ ]:
y.shape, o.shape
y*o[:, None]

In [ ]:
dataset.labels

In [ ]:
H.organization

In [ ]:
sum(H.forward(x)["phylum"])

In [ ]:
x = utils.get_cuda(x)

node = [x for x in H.nodes if x.name=="family_Arachnida"][0]
out = node.net(x)
node.net.compute_prediction(out)

In [ ]:
dataset[1]["label"].shape

In [ ]:
dataset.order

In [ ]:
out = next(iter(testing))
_, label = out["image"], out["label"]

label.shape

In [ ]:
dataset.set_order(["Chordata", "Reptilia"])

In [ ]:
out = next(iter(testing))
_, label = out["image"], out["label"]

label.shape